# Проект: Исследование надёжности заёмщиков

# Оглавление:
- [Ознакомление с данными](#step_1)
- [Предобработка данных](#step_2)
    - [работа с пропусками](#step_2_1)
    - [замена типа данных](#step_2_2)
    - [обработка дубликатов](#step_2_3)
    - [лемматизация](#step_2_4)
    - [категоризация данных](#step_2_5)
- [Ответы на вопросы](#step_3)
    - [вопрос 1](#step_3_1)
    - [вопрос 2](#step_3_2)
    - [вопрос 3](#step_3_3)
    - [вопрос 4](#step_3_4)
- [Общий вывод](#step_4)

# Ознакомление с данными 
<a id="step_1"></a>

Загрузим таблицу, взглянем на неё и посмотрим общую информацию:

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**На первый взгляд увидел несколько проблем в таблице:**
- отрицательные значения в столбце "days_employed";
- разные регистры текста в столбце "education";
- имеются пропуски в столбцах: "days_employed" и "total_income";
- столбцы: "days_employed" и "total_income" не в целых числах.

---

# Предобработка данных
<a id="step_2"></a>

## Обработка пропусков
<a id="step_2_1"></a>

Выведем информацию по пропускам в таблице:

In [3]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Судя по тому, что количество нулевых значений в столбцах стажа и зарплаты равны, возникает предположение, что это люди с нулевым стажем, а соответственно и зарплата отсутствует. Вероятно система сбора данных нулевые значения просто не внесла. <br>
<br>
Проверим: <br>
В возрасте 21-го года не все люди уже имеют стаж работы, соответственно и официальной зарплаты у них может не быть. <br>
Значит, можно сделать сводную таблицу из столбцов зарплаты и стажа, а так же добавить стобец возраста, чтобы по нему сделать выборку.

In [4]:
data_non_employed = data[['dob_years', 'days_employed', 'total_income']]
data_non_employed[data_non_employed['dob_years'] == 21].head(10)

,dob_years,days_employed,total_income
23,21,-272.981385,128265.720871
65,21,NaN,NaN
219,21,-597.273402,86954.922465
317,21,NaN,NaN
606,21,-880.221113,145334.865002
735,21,-1013.920085,56394.236751
763,21,NaN,NaN
1057,21,NaN,NaN
1106,21,-702.043712,197191.199326
1325,21,NaN,NaN


Предположение о том, что пропущенные значения совпадают имеют подтверждение. <br>
В таком случае, считаю вполне целесообразным замену 'NaN' на число 0.

In [5]:
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)
data[data['dob_years'] == 21].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
23,0,-272.981385,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265.720871,сыграть свадьбу
65,0,0.000000,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,0.000000,операции с коммерческой недвижимостью
219,0,-597.273402,21,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,86954.922465,операции с коммерческой недвижимостью
317,0,0.000000,21,высшее,0,Не женат / не замужем,4,M,сотрудник,0,0.000000,сделка с автомобилем
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
735,0,-1013.920085,21,среднее,1,женат / замужем,0,F,сотрудник,0,56394.236751,покупка жилья для сдачи
763,0,0.000000,21,среднее,1,гражданский брак,1,M,компаньон,0,0.000000,покупка жилья
1057,0,0.000000,21,НЕОКОНЧЕННОЕ ВЫСШЕЕ,2,Не женат / не замужем,4,M,компаньон,0,0.000000,операции с недвижимостью
1106,0,-702.043712,21,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,197191.199326,на проведение свадьбы
1325,1,0.000000,21,среднее,1,гражданский брак,1,F,сотрудник,0,0.000000,свадьба


In [6]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь в столбцах стажа и зарплаты вместо пропусков появились числовые значения, которые помогут в дальнейшем.

---

## Замена типа данных
<a id="step_2_2"></a>

В таблице имеются столбцы с дробными числами. Это всё те-же "days_employed" и "total_income". Считаю, что десятые или тысячные части дня или рубля, не изменят общие результаты. Так что перевод этих чисел в целые не коим образом не испортит общую картину, а наоборот упростят подсчёты и вывод результатов. <br>
<br>
Произведём конвертацию:

In [7]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Готово! <br>
Теперь имеем в таблице только цельночисленные значения и строковые.

---

## Обработка дубликатов
<a id="step_2_3"></a>

Проанализируем столбцы с текстом. Посмотрим есть ли в них значения схожие, которые можно объединить в одну категорию.

In [9]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [10]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [11]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [13]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце "education", как было замечено в начале, присутствуют записи в разном регистре, что создаёт дубликаты одних и тех-же категорий. Приведём все записи в один регистр. Это должно исправить проблему.

In [14]:
data['education'] =  data['education'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [15]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь в этом столбце все записи в одинаковом регистре и нет дубликатов.

В столбце "gender" обнаружился "артефакт", человек без указания пола.

In [16]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Если удалить эту строку, то потеряются и другие данные, что может повлиять на результаты. <br>
Считаю более целесобразным заменить значение в столбце на преобладающее. Это не значительно скажется на данных в столбце, но сохранит значения в остальных столбцах.

In [17]:
data['gender'] = data['gender'].replace('XNA', 'F')
data.loc[10701, 'gender']

'F'

In [18]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Когда я делал выборку по возрасту, был обнаружен ещё один "артефакт" в столбце "children", где у 21-го летнего заёмщика указывалось наличие 20-и детей. Стоит проверить эту колонку:

In [19]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

У 76-и человек указано количество детей: 20 и у 47-и: -1. Вероятно это ошибка ввода данных. Посмотрим у кого стоят записи в 20 детей:

In [20]:
data[data['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,-855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,-3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,-2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,-2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
3302,20,0,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,0,профильное образование
3396,20,0,56,высшее,0,женат / замужем,0,F,компаньон,0,0,высшее образование
3671,20,-913,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255,на покупку подержанного автомобиля
3697,20,-2907,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380,на покупку подержанного автомобиля
3735,20,-805,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200,ремонт жилью


Видно, что такие записи ставились абсолютно разным людям.<br>
Ещё заметил новый "артефакт": возраст со значением 0. Вернёмся к нему чуть позже.<br>
Решением вопроса с возрастом вижу два варианта:
- если предположить, что это ошибка при вводе/чтении данных и 20 - это на самом деле 2, а -1 - это скорее всего 1. В таком случае можно заменить "артефакты" на корректные значения;
- можно заменить некорректные значения на среднее число или медиану от всей колонки.

Для начала посмотрим какие значения дадут нам медиана и среднее:

In [21]:
data['children'].mean()

0.5389082462253194

In [22]:
data['children'].median()

0.0

Если воспользоваться медианой, то получится, что у этих людей детей нет и это может повлиять на конечные результаты.<br>
Среднее значение тоже не даёт нам хороших результатов, так как с "половинкой" ребёнка не понятно есть он всё-таки или нет. К тому-же число будет не целое.<br>
В таком случае, считаю целесобразней прибегнуть к первому варианту и заменить значения: 20 на 2 и -1 на 1.

In [23]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Так намного лучше.<br>
Теперь вернёмся к возрасту:

In [24]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Среди всех возрастов "аномальным" является только 0.<br>
В данном случае считаю целесобразным заменить это значение на медиану т.к. среднее значение может дать нецелое число, а от этого мы ранее избавлялись.

In [25]:
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median())

- Одинаковые слова с разными регистрами исправлены.
- "Артефакты" исправлены, основные данные не пострадали.

В остальных колонках схожих по смыслу слов нет, за исключением колонки "purpose", там целесобразнее будет провести лемматизацию и переоформление колонки по категориям на основании ключевых слов.

---

## Лемматизация
<a id="step_2_4"></a>

Для проведения лемматизации откроем библиотеку PyMystem. Произведём замену слов в колонке 'purpose' на их леммы.

In [26]:
from pymystem3 import Mystem
m = Mystem()

data['purpose'] = data['purpose'].apply(m.lemmatize)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]"


Получилось довольно много одинаковых слов в разных записях:

In [27]:
data['purpose'].value_counts()

[автомобиль, \n]                                          973
[свадьба, \n]                                             797
[на,  , проведение,  , свадьба, \n]                       777
[сыграть,  , свадьба, \n]                                 774
[операция,  , с,  , недвижимость, \n]                     676
[покупка,  , коммерческий,  , недвижимость, \n]           664
[операция,  , с,  , жилье, \n]                            653
[покупка,  , жилье,  , для,  , сдача, \n]                 653
[операция,  , с,  , коммерческий,  , недвижимость, \n]    651
[покупка,  , жилье, \n]                                   647
[жилье, \n]                                               647
[покупка,  , жилье,  , для,  , семья, \n]                 641
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        634
[операция,  , со,  , свой,  , недвижимость, \n]           630
[строительство,  , жилой,  , недвижимость, \n]            626
[покупка

Не смотря на всё многообразие строк, можно выделить несколько ключевых слов, например: "Обучение", "Автомобиль" и прочие, которые можно перевести в категории и упростить колонку.

---

## Категоризация данных
<a id="step_2_5"></a>

Чтобы заменить лемматезированные слова в столбце "purpose" создадим функцию, которая при нахождении ключевых слов будет заменять все слова в ячейке на ключевое:

In [28]:
def lem_cat(new_lem):
    if 'жилье' in new_lem:
        return 'недвижимость'
    if 'недвижимость' in new_lem:
        return 'недвижимость'
    if 'автомобиль' in new_lem:
        return 'автомобиль'
    if 'свадьба' in new_lem:
        return 'свадьба'
    if 'образование' in new_lem:
        return 'образование'
    else:
        return 'прочее'

На всякий случай добавил категорию "прочее", на случай если что-то пропустил. <br>
Теперь применим функцию к столбцу:

In [29]:
data['purpose'] = data['purpose'].apply(lem_cat)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


In [30]:
data['purpose'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose, dtype: int64


В итоге получилось всего 4 категории в колонке "purpose". "Прочего" не оказалось, значит ничего не упустил.<br>
<br>
Теперь по тому-же принципу можно создать словари соответствия для колонок "family_status" и "education".

In [31]:
data['education'] = data['education'].apply(m.lemmatize)
data['education'].value_counts()

[средний, \n]                     15233
[высокий, \n]                      5260
[неоконченный,  , высокий, \n]      744
[начальный, \n]                     282
[ученый,  , степень, \n]              6
Name: education, dtype: int64

In [32]:
def lem_edu(new_lem_edu):
    if 'начальный' in new_lem_edu:
        return 'начальное'
    if 'средний' in new_lem_edu:
        return 'среднее'
    if 'неоконченный' in new_lem_edu:
        return 'неоконченное высшее'
    if 'высокий' in new_lem_edu:
        return 'высшее'
    if 'ученый' in new_lem_edu:
        return 'ученая степень'

In [33]:
data['education'] = data['education'].apply(lem_edu)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


In [34]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [35]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [36]:
dict_education = {'среднее': 1, 'высшее': 0, 'неоконченное высшее': 2, 'начальное': 3, 'ученая степень': 4}

Теперь займёмся колонкой "family_status":

In [37]:
data['family_status'] = data['family_status'].apply(m.lemmatize)
data['family_status'].value_counts()

[женатый,  / , замужем, \n]                  12380
[гражданский,  , брак, \n]                    4177
[не,  , женатый,  / , не,  , замужем, \n]     2813
[в,  , развод, \n]                            1195
[вдовец,  / , вдова, \n]                       960
Name: family_status, dtype: int64

In [38]:
def lem_family(new_lem_family):
    if 'не' in new_lem_family:
        return 'холост'
    if 'женатый' in new_lem_family:
        return 'женатый / замужем'
    if 'гражданский' in new_lem_family:
        return 'гражданский брак'
    if 'развод' in new_lem_family:
        return 'в разводе'
    if 'вдовец' in new_lem_family:
        return 'вдовец / вдова'

In [39]:
data['family_status'] = data['family_status'].apply(lem_family)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женатый / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36,среднее,1,женатый / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33,среднее,1,женатый / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32,среднее,1,женатый / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,-2879,43,высшее,0,женатый / замужем,0,F,компаньон,0,240525,недвижимость
7,0,-152,50,среднее,1,женатый / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,-2188,41,среднее,1,женатый / замужем,0,M,сотрудник,0,144425,недвижимость


In [40]:
data['family_status'].value_counts()

женатый / замужем    12380
гражданский брак      4177
холост                2813
в разводе             1195
вдовец / вдова         960
Name: family_status, dtype: int64

In [41]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [42]:
dict_family_status = {'женатый / замужем': 0, 'гражданский брак': 1,
                      'холост': 4, 'в разводе': 3, 'вдовец / вдова': 2}

Теперь таблица пригодна к тому, чтобы проводить вычисления и выводы по категориям.

---

# Ответы на вопросы
<a id="step_3"></a>

## Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id="step_3_1"></a>

Так как вопрос стоит таким образом, что нам надо выяснять не зависимость от количества детей, а от их наличия, то в таком случае можно немного упростить данные в колонке, разделив их на варианты: есть дети, либо нет.
Для начала, сделаем функцию которая будет заменять нулевое значение в столбце "children" на "нет детей" и все остальные на "есть дети", и заменим ими значения в столбце:

In [43]:
def have_children(childrens):
    if childrens == 0:
        return 'нет детей'
    else:
        return 'есть дети'

data['children'] = data['children'].apply(have_children)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,есть дети,-8437,42,высшее,0,женатый / замужем,0,F,сотрудник,0,253875,недвижимость
1,есть дети,-4024,36,среднее,1,женатый / замужем,0,F,сотрудник,0,112080,автомобиль
2,нет детей,-5623,33,среднее,1,женатый / замужем,0,M,сотрудник,0,145885,недвижимость
3,есть дети,-4124,32,среднее,1,женатый / замужем,0,M,сотрудник,0,267628,образование
4,нет детей,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Далее сгруппируем таблицу таким образом, чтобы увидеть количество в каждой категории и сумму не выплаченых кредитов в каждой категории:

In [44]:
data_children_grouped = data.groupby('children').agg({'debt': ['count', 'sum']})
data_children_grouped

debt      
           count   sum
children              
есть дети   7376   678
нет детей  14149  1063

Теперь посчитаем процент по невыплаченым кредитам в каждой категории:

In [45]:
data_children_grouped['процент невозвратов'] = data_children_grouped['debt']['sum'] / data_children_grouped['debt']['count'] * 100
data_children_grouped

debt       процент невозвратов
           count   sum                    
children                                  
есть дети   7376   678            9.191974
нет детей  14149  1063            7.512898

Исходя из того, что получилось можно предположить, что наличие детей усложняет финансовое положение заёмщиков и процент задолженности по кредиту увеличивается. Но разница незначительная, всего чуть больше полутора процентов.
Так что, зависимость есть, но небольшая.

---

## Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id="step_3_2"></a>

В данном вопросе я бы тоже вывел все категории в две:
1. "в браке" - те кто женаты официально и те кто в гражданском браке;
2. "не в браке" - все остальные.

Сперва сверим сходятся-ли колонки "family_status" и "family_status_id" по количеству, заодно станет понятно какая категория к какому id относится:

In [46]:
data['family_status'].value_counts()

женатый / замужем    12380
гражданский брак      4177
холост                2813
в разводе             1195
вдовец / вдова         960
Name: family_status, dtype: int64

In [47]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Далее создадим функцию в которой объединим все текущие категори в две: "в браке" и "не в браке". И после вставим в столбец вместо id:

In [48]:
def family_cat(family_id):
    if family_id == 0:
        return 'в браке'
    if family_id == 1:
        return 'в браке'
    else:
        return 'не в браке'

data['family_status_id'] = data['family_status_id'].apply(family_cat)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,есть дети,-8437,42,высшее,0,женатый / замужем,в браке,F,сотрудник,0,253875,недвижимость
1,есть дети,-4024,36,среднее,1,женатый / замужем,в браке,F,сотрудник,0,112080,автомобиль
2,нет детей,-5623,33,среднее,1,женатый / замужем,в браке,M,сотрудник,0,145885,недвижимость
3,есть дети,-4124,32,среднее,1,женатый / замужем,в браке,M,сотрудник,0,267628,образование
4,нет детей,340266,53,среднее,1,гражданский брак,в браке,F,пенсионер,0,158616,свадьба


Теперь посчитаем количество людей в категориях и количество задолжностей в этих категориях. И после выведем процент по задолжностям в отдельную колонку:

In [49]:
data_family_grouped = data.groupby('family_status_id').agg({'debt': ['count', 'sum']})
data_family_grouped['процент невозвратов'] = data_family_grouped['debt']['sum'] / data_family_grouped['debt']['count'] * 100
data_family_grouped

debt       процент невозвратов
                  count   sum                    
family_status_id                                 
в браке           16557  1319            7.966419
не в браке         4968   422            8.494364

Судя по итоговым результатам, зависимости возврата кредита от семейного положения практически нет.

---

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id="step_3_3"></a>

В данном вопросе можно поступить следующим образом:<br>
Взять среднее значение по всем зарплатам и разделить на две категории: "выше средного" и "ниже среднего".<br>
Для этого создадим функцию, которая разделит уровень доходов на две части, и заменим этими категориями значения в колонке:

In [50]:
def income_level(rub):
    if rub >= data['total_income'].mean():
        return 'выше среднего'
    else:
        return 'ниже среднего'
    
data['total_income'] = data['total_income'].apply(income_level)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,есть дети,-8437,42,высшее,0,женатый / замужем,в браке,F,сотрудник,0,выше среднего,недвижимость
1,есть дети,-4024,36,среднее,1,женатый / замужем,в браке,F,сотрудник,0,ниже среднего,автомобиль
2,нет детей,-5623,33,среднее,1,женатый / замужем,в браке,M,сотрудник,0,ниже среднего,недвижимость
3,есть дети,-4124,32,среднее,1,женатый / замужем,в браке,M,сотрудник,0,выше среднего,образование
4,нет детей,340266,53,среднее,1,гражданский брак,в браке,F,пенсионер,0,выше среднего,свадьба


Далее, так-же как и ранее сгруппируем по сумме в категориях и сумме невозвратов, и выведем проценты:

In [51]:
data_income_grouped = data.groupby('total_income').agg({'debt': ['count', 'sum']})
data_income_grouped['процент невозвратов'] = data_income_grouped['debt']['sum'] / data_income_grouped['debt']['count'] * 100
data_income_grouped

debt       процент невозвратов
               count   sum                    
total_income                                  
выше среднего   9147   719            7.860501
ниже среднего  12378  1022            8.256584

Уровень доходов незначительно влияет на задолженности по кредитам.

---

## Как разные цели кредита влияют на его возврат в срок?
<a id="step_3_4"></a>

Чтобы разобраться в данном вопросе, нам уже не надо ничего переделывать, так как все нужные категории мы вывели ранее.<br>
Осталось лишь сгруппировать таблицу по количеству целей и сумме невозвратов, и вывести проценты:

In [52]:
data_purpose_grouped = data.groupby('purpose').agg({'debt': ['count', 'sum']})
data_purpose_grouped['процент невозвратов'] = data_purpose_grouped['debt']['sum'] / data_purpose_grouped['debt']['count'] * 100
data_purpose_grouped

debt      процент невозвратов
              count  sum                    
purpose                                     
автомобиль     4315  403            9.339513
недвижимость  10840  782            7.214022
образование    4022  370            9.199403
свадьба        2348  186            7.921635

Самые надёжные заёмщики оказались те кто покупает недвижимость, а самые ненадёжные - покупатели автомобилей.

---

# Общий вывод
<a id="step_4"></a>

Для наглядности при подведении итогов, предлагаю создать таблицу с конечными результатами:

In [53]:
final_data = [['есть дети', 9.2],
           ['нет детей', 7.5],
           ['в браке', 8],
           ['не в браке', 8.5],
           ['доход выше среднего', 7.9],
           ['доход ниже среднего', 8.3],
           ['покупка автомобиля', 9.3],
           ['операции с недвижимостью', 7.2],
           ['получение образования', 9.2],
           ['организация свадьбы', 7.9]]
final_columns = ['категория заёмщиков', 'процент должников в категории']
final_result = pd.DataFrame(data=final_data, columns=final_columns)
final_result

,категория заёмщиков,процент должников в категории
0,есть дети,9.2
1,нет детей,7.5
2,в браке,8.0
3,не в браке,8.5
4,доход выше среднего,7.9
5,доход ниже среднего,8.3
6,покупка автомобиля,9.3
7,операции с недвижимостью,7.2
8,получение образования,9.2
9,организация свадьбы,7.9


**Исследование показало следующее:**
- Люди с детьми имеют большую финансовую нагрузку по сравнению с людьми у которых нет детей, вероятно поэтому у них больше просрочек по кредитам.
- Замужние люди вероятно становятся чуть более ответственными и платят за кредиты чуть лучше чем незамужние.
- Высокий доход вероятно влияет положительно на качество выплат по кредитам. Возможно с ростом дохода, появляется больше финансовых возможностей что позволяет лучше справляться с кредитными обязательствами.
- Покупка автомобиля увеличивает финансовую нагрузку на заёмщика затратами на обслуживание авто и страховку. В связи с этим можно предположить, что выплата кредита в срок становится сложнее.
- Оплата обучения тоже может усложниться тем, что во время обучения сложнее зарабатывать и соответственно платить по счетам.
- Свадьба или покупка жилья наоборот заставляет пересмотреть траты, составить семейный бюджет, что в лучшую сторону сказывается на выплатах.

In [54]:
final_result.sort_values('процент должников в категории')

,категория заёмщиков,процент должников в категории
7,операции с недвижимостью,7.2
1,нет детей,7.5
4,доход выше среднего,7.9
9,организация свадьбы,7.9
2,в браке,8.0
5,доход ниже среднего,8.3
3,не в браке,8.5
0,есть дети,9.2
8,получение образования,9.2
6,покупка автомобиля,9.3


Самыми надёжными плательщиками оказались покупатели недвижимости и заёмщики без детей.<br>
В то время как студенты и покупатели автомобилей оказались самыми ненадёжными заёмщиками.<br>
Думаю, кредитному отделу банка эта информация будет весьма полезна.

---